In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.loc[df['Sex']=='female', 'Gender'] = 1
df.loc[df['Sex']!='female', 'Gender'] = 0

In [6]:
X = df[['Age', 'Gender', 'Fare']]
y = df.Survived

In [7]:
X.head()

,Age,Gender,Fare
0,22.0,0.0,7.2500
1,38.0,1.0,71.2833
2,26.0,1.0,7.9250
3,35.0,1.0,53.1000
4,35.0,0.0,8.0500


In [8]:
from sklearn.model_selection import train_test_split
Xtreino, Xval, ytreino, yval = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
from xgboost import XGBClassifier
xg = XGBClassifier()

In [13]:
from sklearn.model_selection import RandomizedSearchCV
import time
param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

eval_metric=['error','auc']
early_stopping_rounds=10,
eval_set= [(Xtreino, ytreino), (Xval, yval)]

results = []
i = 0
# for nome, modelo in modelos.items():
print("\n\n\n------------------------------------------------------------")
print("Treinando")




------------------------------------------------------------
Treinando


In [14]:
# Combinar os parâmetros de cada modelo com cross-validation (10 folds)
rs_xg = RandomizedSearchCV(xg, param_distributions=param_grid, 
                            n_iter=20,
                            n_jobs=-1, 
                            verbose=1, 
                            cv=10, 
                            scoring='recall', 
                            random_state=42)

In [16]:
rs_xg.fit(Xval, yval)
results.append([rs_xg.best_params_, rs_xg.best_score_])
results = pd.DataFrame(results, columns=['Melhores Parâmetros', 'Recall'])
resultados = results.sort_values('Recall', ascending=False)
resultados

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.8s


[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 2 pruned nodes, max_depth=2
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 2 pruned nodes, max_depth=4
[08:22:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 2 pruned nodes, max_depth=3
[

[Parallel(n_jobs=-1)]: Done 169 out of 200 | elapsed:    2.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    2.2s finished
/Users/fabianofilho/miniconda3/envs/ds/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


AttributeError: 'numpy.float64' object has no attribute 'keys'

In [9]:
xg = xgb.XGBClassifier()

In [10]:
clf = xg.fit(Xtreino, ytreino)

In [11]:
p = clf.predict(Xval)

In [12]:
from sklearn.metrics import classification_report

In [13]:
print(classification_report(yval, p))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       157
           1       0.81      0.68      0.74       111

    accuracy                           0.80       268
   macro avg       0.80      0.78      0.79       268
weighted avg       0.80      0.80      0.80       268



In [14]:
teste = pd.read_csv('test.csv')
teste.loc[teste['Sex']=='female', 'Gender'] = 1
teste.loc[teste['Sex']!='female', 'Gender'] = 0
Xteste = teste[['Age', 'Gender', 'Fare']]

In [15]:
Xteste.columns = xg.get_booster().feature_names

In [16]:
y_pred_teste_original = clf.predict(Xteste)
# [:,1]

In [20]:
sub = pd.DataFrame({'PassengerId': teste["PassengerId"], 'Survived': y_pred_teste_original.reshape((y_pred_teste_original.shape[0]))})
print(sub.head(10))

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
5          897         0
6          898         1
7          899         0
8          900         1
9          901         0


In [21]:
sub.to_csv('submission1.csv', index=False)